# Midlertidig lagring av personlig skattemelding med næringspesifikasjon

Denne demoen er ment for å vise hvordan flyten for et sluttbrukersystem kan opprette en midlertidig lagret skattemelding

In [1]:
try:
    from skatteetaten_api import main_relay, base64_decode_response, decode_dokument
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from pathlib import Path
except ImportError as e:
    print("Mangler en eller avhengighet, installer dem via pip, se requierments.txt fil for detaljer")
    raise ImportError(e)


# hjelpe metode om du vil se en request printet som curl
# noinspection PyShadowingNames
def print_request_as_curl(r):
    command = "curl -X {method} -H {headers} -d '{data}' '{uri}'"
    method = r.request.method
    uri = r.request.url
    data = r.request.body
    headers = ['"{0}: {1}"'.format(k, v) for k, v in r.request.headers.items()]
    headers = " -H ".join(headers)
    print(command.format(method=method, headers=headers, data=data, uri=uri))

## Variabler som må endres i test
Før testscriptet kjøres trengs det bare å definere URLer som skal brukes.

In [2]:
midlertidig_lagring_path = "/api/skattemelding/v2/til-midlertidig-lagret-skattemelding-for-visning"

# oppdater med skattemelding-api-relay fra aktuelt testmiljø
env = "https://idporten-api-sbstest.sits.no" # etm2vk-s

# oppdater med test fødselsnummerene du har fått tildelt
fnr = "09845297174"

# oppdater med ønsket innsendingstype - komplett / ikkeKomplett
# typeInnsending = "komplett"
typeInnsending = "ikkeKomplett"

# oppdater med inntektsaar innsendingen gjelder for
inntektsaar = 2022

## Generer ID-porten token
Tokenet er gyldig i 300 sekunder, rekjørt denne biten om du ikke har nådd Altinn3 biten innen 300 sekunder

In [ ]:
idporten_header = main_relay()
s = requests.Session()
s.headers = dict(idporten_header)

### Utkast

In [ ]:
url_utkast = f'{env}/api/skattemelding/v2/utkast/{inntektsaar}/{fnr}'
r = s.get(url_utkast)

In [ ]:
print(r.text)


### Gjeldende

In [ ]:
url_gjeldende = f'{env}/api/skattemelding/v2/{inntektsaar}/{fnr}'
r_gjeldende = s.get(url_gjeldende)
r_gjeldende

#### Fastsatt
Her får en _http 404_ om vedkommende ikke har noen fastsetting, rekjørt denne etter du har sendt inn og fått tilbakemdling i Altinn at den har blitt behandlet, du skal nå ha en fastsatt skattemelding om den har blitt sent inn som Komplett

In [ ]:
url_fastsatt = f'{env}/api/skattemelding/v2/fastsatt/{inntektsaar}/{fnr}'
r_fastsatt = s.get(url_fastsatt)
r_fastsatt

## Svar fra hent gjeldende 

### Gjeldende dokument referanse: 
I responsen på alle api kallene, være seg utkast/fastsatt eller gjeldene, så følger det med en dokumentreferanse. 
For å kalle valider tjenesten, er en avhengig av å bruke korrekt referanse til gjeldende skattemelding. 

Cellen nedenfor henter ut gjeldende dokumentrefranse printer ut responsen fra hent gjeldende kallet 

In [ ]:
sjekk_svar = r_gjeldende

sme_og_naering_respons = xmltodict.parse(sjekk_svar.text)
skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
sme_base64 = skattemelding_base64["content"]
dokref = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]['skattemeldingdokument']['id']
decoded_sme_xml = decode_dokument(skattemelding_base64)
# noinspection PyUnboundLocalVariable
sme_utkast = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()
print(f"Responsen fra hent gjeldende ser slik ut, gjeldende dokumentrerefanse er {dokref}\n")
print(xml.dom.minidom.parseString(sjekk_svar.text).toprettyxml())

sme_dict = xmltodict.parse(decoded_sme_xml['content'])
partsnummer = sme_dict['skattemelding']['partsreferanse']

In [ ]:

if inntektsaar == 2021: 
    with open("../../../src/resources/eksempler/2021/Naeringspesifikasjon-enk-v2.xml", 'r') as f:
        naering_enk_xml = f.read()
elif inntektsaar == 2022:
    with open("../../../src/resources/eksempler/2022/Naeringspesifikasjon-enk-v3.xml", 'r') as f:
        naering_enk_xml = f.read()

innsendingstype = typeInnsending
naeringsspesifikasjoner_enk_b64 = base64.b64encode(naering_enk_xml.encode("utf-8"))
naeringsspesifikasjoner_enk_b64 = str(naeringsspesifikasjoner_enk_b64.decode("utf-8"))
skattemeldingPersonligSkattepliktig_base64=sme_base64 #bruker utkastet uten noen endringer
naeringsspesifikasjoner_base64=naeringsspesifikasjoner_enk_b64
dok_ref=dokref
inntektsaar_innsending=inntektsaar

valider_konvlutt_v2 = """
<?xml version="1.0" encoding="utf-8" ?>
<skattemeldingOgNaeringsspesifikasjonRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:request:v2">
    <dokumenter>
        <dokument>
            <type>skattemeldingPersonlig</type>
            <encoding>utf-8</encoding>
            <content>{sme_base64}</content>
        </dokument>
        <dokument>
            <type>naeringsspesifikasjon</type>
            <encoding>utf-8</encoding>
            <content>{naeringsspeifikasjon_base64}</content>
        </dokument>
    </dokumenter>
    <dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingPersonlig</dokumenttype>
        <dokumentidentifikator>{dok_ref}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
    <inntektsaar>{inntektsaar_innsending}</inntektsaar>
    <innsendingsinformasjon>
        <innsendingstype>{innsendingstype}</innsendingstype>
        <opprettetAv>TurboSkatt</opprettetAv>
    </innsendingsinformasjon>
</skattemeldingOgNaeringsspesifikasjonRequest>
""".replace("\n","")


naering_enk = valider_konvlutt_v2.format(
    sme_base64=skattemeldingPersonligSkattepliktig_base64,
    naeringsspeifikasjon_base64=naeringsspesifikasjoner_base64,
    dok_ref=dok_ref,
    inntektsaar_innsending=inntektsaar_innsending,
    innsendingstype=innsendingstype)

# Valider utkast sme med næringsopplysninger

In [ ]:
def valider_sme(payload):
    url_valider = f'{env}/api/skattemelding/v2/valider/{inntektsaar}/{fnr}'
    header = dict(idporten_header)
    header["Content-Type"] = "application/xml"
    return s.post(url_valider, headers=header, data=payload)


valider_respons = valider_sme(naering_enk)
resultatAvValidering = xmltodict.parse(valider_respons.text)["skattemeldingOgNaeringsspesifikasjonResponse"]["resultatAvValidering"]

if valider_respons:
    print(resultatAvValidering)
    print()
    print(xml.dom.minidom.parseString(valider_respons.text).toprettyxml())
else:
    print(valider_respons.status_code, valider_respons.headers, valider_respons.text)

## Midlertidig lagret skattemelding
I dette steget sender vi inn en skattemelding-xml som er opprettet av et sluttbrukersystem.

In [ ]:
header = dict(idporten_header)
header["Content-Type"] = "application/xml"
header["Korrelasjonsid"] = "319f5c9a-17e4-4e8e-a399-5a4899ccd84d"
r = s.post(f"{env}{midlertidig_lagring_path}", data=naering_enk, headers=header)
r

### Print link til midlertidig lagret skattemelding

In [ ]:
print(r.json()['url'])